# 以Transformers套件实作问答(Question Answering)功能

## 载入相关套件

In [1]:
from transformers import pipeline

## 载入模型

In [2]:
nlp = pipeline("question-answering")

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


## 训练资料

In [3]:
context = r"Extractive Question Answering is the task of extracting an answer " + \
"from a text given a question. An example of a question answering " + \
"dataset is the SQuAD dataset, which is entirely based on that task. " + \
"If you would like to fine-tune a model on a SQuAD task, you may " + \
"leverage the examples/question-answering/run_squad.py script."

## 测试 2 笔资料

In [4]:
result = nlp(question="What is extractive question answering?", context=context)
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}",
        f", start: {result['start']}, end: {result['end']}")

print()

result = nlp(question="What is a good example of a question answering dataset?", 
             context=context)
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}",
        f", start: {result['start']}, end: {result['end']}")

Answer: 'the task of extracting an answer from a text given a question', score: 0.6226 , start: 33, end: 94

Answer: 'SQuAD dataset', score: 0.5053 , start: 146, end: 159


## 结合Tokenizer

In [5]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

# 结合分词器(Tokenizer)
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

## 训练资料

In [6]:
text = r"""
🤗 Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose
architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural
Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between
TensorFlow 2.0 and PyTorch.
"""

## 问题

In [7]:
questions = [
    "How many pretrained models are available in 🤗 Transformers?",
    "What does 🤗 Transformers provide?",
    "🤗 Transformers provides interoperability between which frameworks?",
]

## 推测答案

In [8]:
for question in questions:
    inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]

    outputs = model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    # Get the most likely beginning of answer with the argmax of the score
    answer_start = torch.argmax(answer_start_scores)
    # Get the most likely end of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1

    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end])
    )

    print(f"Question: {question}")
    print(f"Answer: {answer}")

Question: How many pretrained models are available in 🤗 Transformers?
Answer: over 32 +
Question: What does 🤗 Transformers provide?
Answer: general - purpose architectures
Question: 🤗 Transformers provides interoperability between which frameworks?
Answer: tensorflow 2. 0 and pytorch
